In [1]:
import numpy as np
import pandas as pd

### Initialize Parameters

In [2]:
# Xavier initialization is poor for relu

def initialize_params(dims): # He initialization ; for ReLu and Leaky ReLu activations
    """
    Arguments:
    dims - python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters - python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1]) 
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(0)
    params = {}
    for i in range(1, len(dims)):
        params.update({f'W{i}': np.random.randn(dims[i],dims[i-1]) * np.sqrt(2 / dims[i])}) # scaled initialization for ReLU
        params.update({f'B{i}': np.zeros((dims[i], 1))})

    assert(params['W' + str(i)].shape == (dims[i], dims[i-1])) # double check dimensions are correct
    assert(params['B' + str(i)].shape == (dims[i], 1))

    return params

### Define activation functions

Using ReLu for this assignment. 

Several others are included which I would like to fill in later for my own understanding/research

In [3]:
"""
Args:
Z - pre activation parameter ; output of the linear layer

Returns:
A - post activation parameter ; of the same shape as Z
cache - returns Z ; used during backpropagation
"""

def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    cache = Z
    return A, cache

def sigmoid_backward(dA, cache):
    Z = cache
    s = 1 / (1 + np.exp(-Z)) # the sigmoid function
    dZ = dA * (s * (1-s)) # dA * the derivative of sigmoid. sigmoid derivative = (e ** -x) /(1 + e ** -X) = sig(x) * (1 - sig(x))

    assert (dZ.shape == Z.shape)

    return dZ

def relu(Z):
    A = np.maximum(0,Z)
    cache = Z
    return A, cache

def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    dZ[Z <= 0] = 0 # When z <= 0, you should set dz to 0 as well - think of what ReLu looks like. 

    assert (dZ.shape == Z.shape) # check if dZ has correct shape
    
    return dZ

def tanh(Z):
    A = (np.exp(Z) - np.exp(-Z)) / (np.exp(Z) + np.exp(-Z))
    return A, cache

def tanh_backward(dA, cache):
    Z = cache
    t = (np.exp(Z) - np.exp(-Z)) / (np.exp(Z) + np.exp(-Z)) # the tanh function
    dZ = dA * (1 - t ** 2) # dA * dervative of tanh. 

    assert (dZ.shape == Z.shape) # check if dZ has correct shape

    return dZ

def softplus(Z):
    A = np.log(1 + np.exp(Z))
    cache = Z
    return A, cache

def softplus_backward(dA, cache): # define later
    pass

def softmax(Z): # really only used in the output layer
    A = np.exp(Z - np.max(Z, axis=1, keepdims=True)) / np.exp(Z - np.max(Z, axis=1, keepdims=True)).sum(axis=1, keepdims=True) # numerically stable softmax
    cache = Z
    return A, cache

def softmax_backeard(dA, cache): # define later
    pass


### Feed Forward Method

In [4]:
def linear_forward (A, W, b): # linear transformation
    """
    Args:
    A - activations from previous layer: numpy array of shape (size of previous layer, number of examples)
    W - weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b - bias vector: numpy array of shape (size of current layer, 1)

    Returns:
    Z - input of the activation function (pre-activation parameter)
    cache - tuple: (A, W, b) ; used for computing backwards pass
    """
    
    Z = np.dot(W, A) + b  
    cache = (A, W, b) 

    assert(Z.shape == (W.shape[0], A.shape[1])) # double check dimensions are correct (W rows, A columns)

    return Z, cache

In [5]:
def linear_activation_forward(A_prev, weights, biases, activation): # linear transformation with activation function
    """
    Args:
    A_prev - activations from previous layer: numpy array of shape (size of previous layer, number of examples)
    weights - weights matrix: numpy array of shape (size of current layer, size of previous layer)
    biases - bias vector: numpy array of shape (size of current layer, 1)
    activation - the activation function to be used: string

    Returns:
    A_new - output of activation function: 
    cache - tuple: (linear cache, activation cache) ; used for computing backwards pass
    """

    
    Z, linear_cache = linear_forward(A_prev, weights, biases) # calculate pre-activation parameter from previous inputs, weight matrix and biases

    if activation == "tanh":
        A_new, activation_cache = tanh(Z)

    elif activation == "sigmoid":
        A_new, activation_cache = sigmoid(Z)

    elif activation == "relu":
        A_new, activation_cache = relu(Z)

    elif activation == "softplus":
        pass

    assert (A_new.shape == (weights.shape[0] , A_prev.shape[1])) # (W rows, A cols)

    cache = (linear_cache, activation_cache) # entry 1: tuple containg A, W, b ; entry 2: Z that was used in activation function

    return A_new, cache

In [6]:
def model_forward(X, parameters): # feed forward
    """
    Args:
    X - data to be modeled: numpy array of shape (input size, number of examples)
    parameters - python dictionary containing parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)

    Returns:
    A_final - activations from final layer
    caches - an indexed list of caches from every layer containing the previous activations, weights, biases, and pre-activation parameters as tuples 
    """

    caches = [] 
    A = X
    L = len(parameters) // 2 # params accounts for weight matrices and bias vectors (2 things)

    for i in range(1, L): # linear --> relu for layers 1-->(L-1)
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters[f'W{i}'], parameters[f'B{i}'], "relu") # hidden layers # note: relu is hard coded
        caches.append(cache)

    # linear transformation for last layer (regression model)
    A_L, linear_cache = linear_forward(A, parameters[f'W{L}'], parameters[f'B{L}']) # output layer # note: for regression, usually no activation (or a linear transformation) is used.
    caches.append((linear_cache, None))

    assert(A_L.shape == (1, X.shape[1])) # double check dimensions are correct

    return A_L, caches

### Define some loss functions

Will be using MSE in this assignment  
Several others are included which I would like to fill in later for my own understanding/research  
### Dot Product Formula

The dot product of two vectors $\mathbf{A}$ and $\mathbf{B}$ is given by:

$$
\mathbf{A} \cdot \mathbf{B} = \sum_{i=1}^{n} A_i \cdot B_i
$$

### Mean Squared Error (MSE) Formula

The Mean Squared Error (MSE) is calculated as:

$$
MSE = \frac{1}{m} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

### Gradient of MSE

The gradient of MSE is calculated as 

$$
dA_L = \frac{2}{m} \ (A_L-Y)
$$

where:  
$A_L$ is the matrix of predictions (output activations)   
$Y$ is the matrix of true labels (ground truth)   
$m$ is the number of examples (data points)

In [161]:
"""
Args:
AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

Returns:
cost -- cross-entropy cost
"""

def binary_cross_entropy(AL, Y): # log loss ; classification
    m = Y.shape[1]
    cost = (-1/m) * (np.dot(Y, np.log(AL).T) + np.dot((1-Y), np.log(1-AL).T)) # using dot product instead of np.sum() becasue it's faster
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

def categorical_cross_entropy(AL, Y): # log loss ; classification
    m = Y.shape[1]
    cost = (-1/m) * np.sum(np.mulitply(Y, np.log(AL))) # this is unfinished and I'm not sure if it's correct

    return cost

def neg_log_likelihood(): # for simple classifications
    pass

def MSE(AL, Y): # mean squared error ; regression
    #m = Y.shape[1]
    m = Y.shape[0]
    cost = (1 / m) * np.dot((AL-Y).flatten(), (AL-Y).flatten()) # using dot product instead of np.sum() because it's faster
                                                                # does anything need to be trasposed?
    return cost

def MAE(): # mean absolute error ; regression
    pass

def KL_divergence(): # Kullback-Leibler Divergence
    pass

def Huber(): # Huber loss ; regression
    pass

def MSE_grad(AL, Y):
    m = Y.shape[0]
    dAL = (2/m) * (AL - Y)
    return dAL

### Define a backprop method

In [203]:
# can the autodiff assignment be used here?

def linear_backward(dZ, cache):
    """
    Args:
    dZ - Gradient of the cost with respect to the linear output (of the current layer)
    cache - tuple ; (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev - Gradient of the cost with respect to the activation (of the previous layer) ; same shape as A_prev
    dW - Gradient of the cost with respect to W (of the current layer) ; same shape as W
    db - Gradient of the cost with respect to b (of the current layer) ; same shape as b
    """

    A_prev, W, b = cache
    m = A_prev.shape[1] # is this the length of entries?


    dA_prev = np.dot(W.T, dZ) # what's happening here?
    dW = (1/m) * np.dot(dZ, A_prev.T) # what's happening here?
    db = (1/m) * np.sum(dZ, axis = 1, keepdims = True) # what's happening here?

    assert (dA_prev.shape == A_prev.shape) 
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [9]:
def linear_activation_backward(dA, cache, activation):
    """
    activation function takes input Z and returns A and an activation cache (Z) ; 
    activation_backward function takes input dA and the activation cache (Z) and returns dZ.
    
    Args:
    dA - post-activation gradient for current layer
    cache - tuple ; (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation - activation function used in this layer

    Returns:
    dA_prev - Gradient of the cost with respect to the activation (of the previous layer) ; same shape as A_prev
    dW - Gradient of the cost with respect to W (of the current layer) ; same shape as W
    db - Gradient of the cost with respect to b (of the current layer) ; same shape as b
    """

    linear_cache, activation_cache = cache

    if activation == "relu": # relu activation takes input Z and returns A and an activation cache (Z) ; relu_backward takes input dA and the activation cache (Z) and returns dZ
        dZ = relu_backward(dA, activation_cache)

    elif activation == "tanh":
        dZ = tanh_backward(dA, activation_cache)

    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)

    dA_prev, dW, b = linear_backward(dZ, linear_cache)

    return dA_prev, dW, b

### Updated linear_activation_backward() function which accounts for not having an activation cache in the final layer

In [10]:
def linear_activation_backward(dA, cache, activation = None):
    """
    activation function takes input Z and returns A and an activation cache (Z) ; 
    activation_backward function takes input dA and the activation cache (Z) and returns dZ.
    
    Args:
    dA - post-activation gradient for current layer
    cache - tuple ; (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation - activation function used in this layer

    Returns:
    dA_prev - Gradient of the cost with respect to the activation (of the previous layer) ; same shape as A_prev
    dW - Gradient of the cost with respect to W (of the current layer) ; same shape as W
    db - Gradient of the cost with respect to b (of the current layer) ; same shape as b
    """

    linear_cache, activation_cache = cache

    if activation == "relu": # relu activation takes input Z and returns A and an activation cache (Z) ; relu_backward takes input dA and the activation cache (Z) and returns dZ
        dZ = relu_backward(dA, activation_cache)

    elif activation == "tanh":
        dZ = tanh_backward(dA, activation_cache)

    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)

    elif activation is None: # No activation function ; final layer for regression
        dZ = dA # Z = A and dZ = dA for the final layer if no activation is used

    dA_prev, dW, b = linear_backward(dZ, linear_cache)

    return dA_prev, dW, b

In [194]:
def model_backward(AL, Y, caches):
    """
    Args:
    AL - probability vector, output of the forward propagation (L_model_forward())
    Y - true "label" vector
    caches - list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
                how would this work for tanh? softplus?

    Returns:
    grads - dictionary ; gradients with respect to activations, weights, biases...
                         grads["dA" + str(l)] = ... 
                         grads["dW" + str(l)] = ...
                         grads["db" + str(l)] = ... 
    """

    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    # m = AL.shape[0]
    Y = Y[0]
    # Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL

    # dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # initialize backprop, note: this is the gradient of cross entropy loss
    dAL = MSE_grad(AL, Y) # gradient for mean-squared-error

    current_cache = caches[L-1] # Last Layer
    #current_cache = caches[L-2] # b/c no activation cache in last layer since no activation function was used
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(
        dAL, 
        current_cache, 
        activation = None) # note: no activation in the final layer for regression ; this is hard coded

    for i in reversed(range(L-1)): # Loop from l=L-2 to l=0
        current_cache = caches[i]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(
            grads["dA" + str(i+1)],
            current_cache,
            activation = "relu") # note: relu is hard coded ; these are the hidden layers

        grads["dA" + str(i)] = dA_prev_temp
        grads["dW" + str(i+1)] = dW_temp
        grads["db" + str(i+1)] = db_temp

    return grads

In [59]:
print(A_L.T.shape)
print(Y_train.shape)

(404, 1)
(404,)


### Update Parameters
which gradient descent update rule to use? 

which learning rate to use?

In [214]:

def update_params(params, grads, learning_rate):
    """
    generally we update by: new_params = params - grads * learning_rate
    Args:
    params - dict ; contains the (previous) parameters
    grads - dict ; contains the graduients calculated from backprop
    learning_rate - step size for gradient descent
    
    Returns:
    params - dict ; the updated parameters
                parameters["W" + str(l)] = ... 
                parameters["b" + str(l)] = ...
    """

    L = len(params) // 2 # params dict contains entries for both weights and biases

    for i in range(L):
        params["W" + str(i+1)] = params["W" + str(i+1)] - grads["dW" + str(i+1)] * learning_rate
        params["B" + str(i+1)] = params["B" + str(i+1)] - grads["db" + str(i+1)] * learning_rate

    return params

### Import a Dataset for regression

There are 14 attributes in each case of the dataset. They are:  
CRIM - per capita crime rate by town  
ZN - proportion of residential land zoned for lots over 25,000 sq.ft.  
INDUS - proportion of non-retail business acres per town.  
CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)  
NOX - nitric oxides concentration (parts per 10 million)  
RM - average number of rooms per dwelling  
AGE - proportion of owner-occupied units built prior to 1940  
DIS - weighted distances to five Boston employment centres  
RAD - index of accessibility to radial highways  
TAX - full-value property-tax rate per ${$10,000} $  
PTRATIO - pupil-teacher ratio by town  
B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town  
LSTAT - % lower status of the population  
MEDV - Median value of owner-occupied homes in \$1000's  

In [15]:
Housing = pd.read_csv("BostonHousing.csv")
Housing.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [16]:
# put the data set into numpy. Goal is to predict the age of a person based on the other attributes for housing

features = Housing.columns.drop('age').to_list() 

inputs = Housing[features].to_numpy() # input features
targets = Housing.age.to_numpy() # target variables

In [159]:
# split the data set up into training sets and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(inputs, targets, test_size = 0.2, random_state = 0)

Y_train = Y_train.reshape(1,-1) # reshape Y_train to be the same dimensions as A_L (1,404)
X_train = X_train.T # transpose to have 13 rows (for features) and 406 columns (for data points)
X_test = X_test.T # this makes things compatible with the layout of the weights matrix

### Set up neural network

In [116]:
# set the number layers and nodes in each
# initialize parameters

dimensions = [13,8,5,1] # the layout of the neural network
# input layer must have dims of # of features. output layer must have dim of 1 for regression (multiple for multiclass classification)

params = initialize_params(dimensions) # initial weights and biases

In [204]:
A_L, cache_forward = model_forward(X_train, params)

In [205]:
assert A_L.shape == Y_train.shape
error = MSE(A_L, Y_train)
print(error)

283894858.8188946


In [206]:
dAL = MSE_grad(A_L, Y_train)

In [207]:
grads = model_backward(A_L, Y_train, cache_forward)

In [216]:
new_params = update_params(params, grads, learning_rate=0.01)

### Loop through the process to learn the model